# Interactive visualization of the loaded graph

## set up environment

Load the library dependencies

In [1]:
import json
import pathlib

import networkx as nx
import pyvis
import watermark

Optional: filter the less useful warnings

In [2]:
#warnings.filterwarnings("ignore")

Show a watermark of the OS, hardware, language environment, and dependent library versions.

In [3]:
%load_ext watermark
%watermark
%watermark --iversions

Last updated: 2025-02-05T13:16:36.482448-08:00

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 8.31.0

Compiler    : Clang 13.0.0 (clang-1300.0.29.30)
OS          : Darwin
Release     : 23.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

watermark: 2.5.0
pyvis    : 0.3.2
networkx : 3.4.2
json     : 2.0.9



## visualize results

In [4]:
def render_graph (
    graph: nx.DiGraph,
    ) -> pyvis.network.Network:
    """
    Generate a `PyVis` network visualization from the DiGraph
    """
    net: pyvis.network.Network = pyvis.network.Network(
        notebook = True,
        cdn_resources = "in_line",
        height = "700px", 
        width = "100%",
    )

    # calculate the _betweenness centrality_ measures for each country
    cent: dict = nx.betweenness_centrality(graph, weight = "weight")

    for src_id, dat in graph.nodes(data = True):
        graph.nodes[src_id]["value"] = cent[src_id]

    # load the network and adjust attributes for visualization
    net.from_nx(graph)

    for node in net.nodes:
        node["label"] = node["name"]

    for edge in net.edges:
        #edge["value"] = edge["width"]
        pass

    return net

In [5]:
graph: nx.DiGraph = nx.DiGraph

graph_file: pathlib.Path = pathlib.Path("graph.json")

with open(graph_file, "r", encoding = "utf-8") as fp:
    graph = nx.node_link_graph(
        json.load(fp),
        edges = "edges",
    )

In [6]:
net: pyvis.network.Network = render_graph(graph)

net.force_atlas_2based()
net.show_buttons(filter_ = ["physics"])

#net.toggle_physics(False)
net.show("networkx-pyvis.html")

networkx-pyvis.html


Note: to see the final graph, load the HTML file `network-pyvis.html` in your browser.